In [1]:
# MLM 방식을 이용한 Further pre-traning 방식 구현 예제
# 참고 소스 : https://towardsdatascience.com/masked-language-modelling-with-bert-7d49793e5d2c 참조 바람
import torch
import os

from tqdm.notebook import tqdm
from transformers import DistilBertTokenizer, BertConfig, DistilBertForMaskedLM
from transformers import AdamW, get_linear_schedule_with_warmup

import sys
sys.path.append("..")
from myutils import GPU_info, seed_everything, mlogging, MLMDatasetbyDistilBert, AccuracyForMaskedToken, SaveBERTModel

In [2]:
# 훈련시킬 말뭉치(사전 만들때 동일한 말뭉치 이용)
#input_corpus = "../../korpora/mycorpus/bong_corpus_mecab.txt"
input_corpus = "../../data11/my_corpus/my/pre-kowiki-20220620-1줄.txt"
#input_corpus = "../../korpora/kowiki_20190620/wiki_20190620_mecab_false_0311.txt"

# eval 말뭉치 
eval_corpus = "../../data11/my_corpus/bong_small_eval.txt"

# 기존 사전훈련된 모델
#model_path = "../../data11/model/distilbert/distilbert-base-multilingual-cased/"
model_path = "../../data11/model/distilbert/dbmc-kowiki-202206-1line/7-22-6-43/"



# 기존 사전 + 추가된 사전 파일
vocab_path="../../data11/model/distilbert/distilbert-base-multilingual-cased/kowiki-202206-1line-vocab/"

# 출력
OUTPATH = '../../data11/model/distilbert/dbmc-kowiki-202206-1line/'

batch_size = 32
token_max_len = 128

device = GPU_info()
print(device)

#seed 설정
seed_everything(333)

#logging 설정
logger =  mlogging(loggername="distilbertfpt-1", logfilename="../../log/distilbertfpt-1")

True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
cuda:0
logfilepath:../../log/distilbertfpt-1_2022-07-22.log


In [3]:
# tokeinzier 생성
# tokenizer 생성
# => BertTokenizer, BertTokenizerFast 둘중 사용하면됨

#tokenizer = DistilBertTokenizer(vocab_file=vocab_path, max_len=token_max_len, do_lower_case=False)
tokenizer = DistilBertTokenizer.from_pretrained(vocab_path, max_len=token_max_len, do_lower_case=False)
#tokenizer = BertTokenizerFast(vocab_speical_path)
# tokenizer = BertTokenizerFast(vocab_file=vocab_file, max_len=token_max_len, do_lower_case=False)


# speical 토큰 계수 + vocab 계수 - 이미 vocab에 포함된 speical 토큰 계수(5)
vocab_size = len(tokenizer.all_special_tokens) + tokenizer.vocab_size - 5 + 1
#vocab_size = len(tokenizer.all_special_tokens) + tokenizer.vocab_size - 5
print('special_token_size: {}, tokenizer.vocab_size: {}'.format(len(tokenizer.all_special_tokens), tokenizer.vocab_size))
print('vocab_size: {}'.format(vocab_size))
print('tokenizer_len: {}'.format(len(tokenizer)))

# 모델 로딩 further pre-training 
#config = BertConfig.from_pretrained(model_path)
model = DistilBertForMaskedLM.from_pretrained(model_path, from_tf=bool(".ckpt" in model_path)) 
#model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')    

#################################################################################
# 모델 embedding 사이즈를 tokenizer 크기 만큼 재 설정함.
# 재설정하지 않으면, 다음과 같은 에러 발생함
# CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)` CUDA 에러가 발생함
#  indexSelectLargeIndex: block: [306,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
#
#     해당 오류는 기존 Embedding(8002, 768, padding_idx=1) 처럼 입력 vocab 사이즈가 8002인데,
#     0~8001 사이를 초과하는 word idx 값이 들어가면 에러 발생함.
#################################################################################
model.resize_token_embeddings(len(tokenizer))

model.to(device)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.


special_token_size: 5, tokenizer.vocab_size: 146443
vocab_size: 146444
tokenizer_len: 146443


DistilBertForMaskedLM(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(146443, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): 

In [4]:
from torch.utils.data import DataLoader, RandomSampler

# true이면, 무조건 원본파일 읽고, cache 파일을 만듬.
# False로 하면 cache파일이 있으면 cache파일 이용함. cache파일 없으면 원본파일 일고, cache파일은 만들지 않음
overwrite_cache = False

# 각 스페셜 tokenid를 구함
CLStokenid = tokenizer.convert_tokens_to_ids('[CLS]')
SEPtokenid = tokenizer.convert_tokens_to_ids('[SEP]')
UNKtokenid = tokenizer.convert_tokens_to_ids('[UNK]')
PADtokenid = tokenizer.convert_tokens_to_ids('[PAD]')
MASKtokenid = tokenizer.convert_tokens_to_ids('[MASK]')
print('CLSid:{}, SEPid:{}, UNKid:{}, PADid:{}, MASKid:{}'.format(CLStokenid, SEPtokenid, UNKtokenid, PADtokenid, MASKtokenid))

#===============================================================================
# eval dataloader 생성
eval_dataset = MLMDatasetbyDistilBert(corpus_path = eval_corpus,
                        tokenizer = tokenizer, 
                        CLStokeinid = CLStokenid ,   # [CLS] 토큰 id
                        SEPtokenid = SEPtokenid ,    # [SEP] 토큰 id
                        UNKtokenid = UNKtokenid ,    # [UNK] 토큰 id
                        PADtokenid = PADtokenid,    # [PAD] 토큰 id
                        Masktokenid = MASKtokenid,   # [MASK] 토큰 id
                        max_sequence_len=token_max_len,  # max_sequence_len)
                        mlm_probability=0.15,
                        overwrite_cache=False
                        )


# eval dataloader 생성
# => tenosor로 만듬
eval_loader = DataLoader(eval_dataset, 
                         batch_size=batch_size, 
                         #shuffle=True, # dataset을 섞음
                         sampler=RandomSampler(eval_dataset, replacement=False), #dataset을 랜덤하게 샘플링함
                         num_workers=4
                        )
#===============================================================================



# distilberttoknizer에는 token_type_ids(문장구분자) 가 없음
# 따라서 MLMDatasetbyDistilBert 함수를 이용하여 MLM 생성함
train_dataset = MLMDatasetbyDistilBert(corpus_path = input_corpus,
                           tokenizer = tokenizer, 
                           CLStokeinid = CLStokenid ,   # [CLS] 토큰 id
                           SEPtokenid = SEPtokenid ,    # [SEP] 토큰 id
                           UNKtokenid = UNKtokenid ,    # [UNK] 토큰 id
                           PADtokenid = PADtokenid,    # [PAD] 토큰 id
                           Masktokenid = MASKtokenid,   # [MASK] 토큰 id
                           max_sequence_len=token_max_len,  # max_sequence_len)
                           mlm_probability=0.15,
                           overwrite_cache=overwrite_cache
                          )


# 학습 dataloader 생성
# => tenosor로 만듬
train_loader = DataLoader(train_dataset, 
                          batch_size=batch_size, 
                          #shuffle=True, # dataset을 섞음
                          sampler=RandomSampler(train_dataset, replacement=False), #dataset을 랜덤하게 샘플링함
                          num_workers=4
                         )

print(train_dataset[0])

CLSid:101, SEPid:102, UNKid:100, PADid:0, MASKid:103
*corpus:../../data11/my_corpus/bong_small_eval.txt
*max_sequence_len:128
*mlm_probability:0.15
*CLStokenid:101, SEPtokenid:102, UNKtokenid:100, PADtokeinid:0, Masktokeid:103
*total_line: 200


  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

*corpus:../../data11/my_corpus/my/pre-kowiki-20220620-1줄.txt
*max_sequence_len:128
*mlm_probability:0.15
*CLStokenid:101, SEPtokenid:102, UNKtokenid:100, PADtokeinid:0, Masktokeid:103
*total_line: 4388544


  0%|          | 0/4388544 [00:00<?, ?it/s]

  0%|          | 0/4388544 [00:00<?, ?it/s]

{'input_ids': tensor([   101,    103,  10003, 120044,  10530, 119860,    103,    217, 122839,
           217, 122791,  10459,    103,  11467,   9641,    103,    102,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,   

In [5]:
##################################################
epochs = 4            # epochs
learning_rate = 2e-5  # 학습률
##################################################

# optimizer 적용
optimizer = AdamW(model.parameters(), 
                 lr=learning_rate, 
                 eps=1e-8) # 0으로 나누는 것을 방지하기 위한 epsilon 값(10^-6 ~ 10^-8 사이 이값 입력합)

# 총 훈련과정에서 반복할 스탭
total_steps = len(train_loader)*epochs
warmup_steps = total_steps * 0.1 #10% of train data for warm-up

# 손실률 보여줄 step 수
p_itr = int(len(train_loader)*0.1)  
    
# step마다 모델 저장
save_steps = int(total_steps * 0.2)
    
# 스캐줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=warmup_steps, 
                                            num_training_steps=total_steps)

itr = 1
total_loss = 0
total_len = 0
total_correct = 0
total_test_correct = 0
total_test_len = 0
            
list_train_loss = []
list_train_acc = []
list_validation_acc = []

model.zero_grad()# 그래디언트 초기화
for epoch in tqdm(range(epochs)):

    model.train() # 훈련모드로 변환
    for data in tqdm(train_loader):
    
        #optimizer.zero_grad()
        model.zero_grad()# 그래디언트 초기화
        
        # 입력 값 설정
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        #token_type_ids = data['token_type_ids'].to(device)         
        labels = data['labels'].to(device)
        #print('Labels:{}'.format(labels))
        
        # 모델 실행
        outputs = model(input_ids=input_ids, 
                        attention_mask=attention_mask,
                        #token_type_ids=token_type_ids,
                        labels=labels)
        
        # 출력값 loss,logits를 outputs에서 얻어옴
        loss = outputs.loss
        logits = outputs.logits
        #print('Loss:{}, logits:{}'.format(loss, logits))
        
        # optimizer 과 scheduler 업데이트 시킴
        loss.backward()   # backward 구함
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   # 그래디언트 클리핑 (gradient vanishing이나 gradient exploding 방지하기 위한 기법)
        optimizer.step()  # 가중치 파라미터 업데이트(optimizer 이동)
        scheduler.step()  # 학습률 감소
        
        # ***further pretrain 에는 손실률 계산을 넣지 않음
        # 정확도 계산하는 부분은 no_grade 시켜서, 계산량을 줄임.
        
        # => torch.no_grad()는 gradient을 계산하는 autograd engine를 비활성화 하여 
        # 필요한 메모리를 줄이고, 연산속도를 증가시키는 역활을 함
        with torch.no_grad():
                        
            # 손실(loss) 계산 
            total_loss += loss.item()
                
            #===========================================
            # 정확도(Accurarcy) 계산
            correct, masked_len = AccuracyForMaskedToken(logits, labels, input_ids, MASKtokenid)           
            total_correct += correct.sum().item() 
            total_len += masked_len 
            #=========================================
         
            # 주기마다 test(validataion) 데이터로 평가하여 손실류 계산함.
            if itr % p_itr == 0:
                train_loss = total_loss/p_itr
                train_acc = total_correct/total_len
                         
                ####################################################################
                # 주기마다 eval(validataion) 데이터로 평가하여 손실류 계산함.
                # 평가 시작
                model.eval()

                #for data in tqdm(eval_loader):
                for eval_data in eval_loader:
                    # 입력 값 설정
                    input_ids = eval_data['input_ids'].to(device)
                    attention_mask = eval_data['attention_mask'].to(device)
                    labels = eval_data['labels'].to(device)

                    # 손실률 계산하는 부분은 no_grade 시켜서, 계산량을 줄임.
                    # => torch.no_grad()는 gradient을 계산하는 autograd engine를 비활성화 하여 
                    # 필요한 메모리를 줄이고, 연산속도를 증가시키는 역활을 함
                    with torch.no_grad():
                        # 모델 실행
                        outputs = model(input_ids=input_ids, 
                                       attention_mask=attention_mask,
                                       labels=labels)

                        # 출력값 loss,logits를 outputs에서 얻어옴
                        #loss = outputs.loss
                        logits = outputs.logits

                        #===========================================
                        # 정확도(Accurarcy) 계산
                        correct, masked_len = AccuracyForMaskedToken(logits, labels, input_ids, MASKtokenid)           
                        total_test_correct += correct.sum().item() 
                        total_test_len += masked_len 
                        #=========================================

                val_acc = total_test_correct/total_test_len
                    
                logger.info('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Train Acc: {:.4f}, Val Acc:{}({}/{})'.format(epoch+1, epochs, itr, train_loss, train_acc, val_acc, total_test_correct, total_test_len))
                    
                list_train_loss.append(train_loss)
                list_train_acc.append(train_acc)
                list_validation_acc.append(val_acc)
                 
                total_loss = 0
                total_len = 0
                total_correct = 0
                total_test_correct = 0
                total_test_len = 0
                ####################################################################

            # 모델 저장
            if itr % save_steps == 0:
                SaveBERTModel(model, tokenizer, OUTPATH, epochs, learning_rate, batch_size)

        itr+=1
   

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/137142 [00:00<?, ?it/s]

2022-07-23 08:14:08,027 - distilbertfpt-1 - INFO - [Epoch 1/4] Iteration 13714 -> Train Loss: 0.0860, Train Acc: 0.5058, Val Acc:0.40202391904323825(437/1087)
2022-07-23 08:58:34,917 - distilbertfpt-1 - INFO - [Epoch 1/4] Iteration 27428 -> Train Loss: 0.0697, Train Acc: 0.5795, Val Acc:0.38822447102115915(422/1087)
2022-07-23 09:43:07,292 - distilbertfpt-1 - INFO - [Epoch 1/4] Iteration 41142 -> Train Loss: 0.0699, Train Acc: 0.5783, Val Acc:0.38546458141674333(419/1087)
2022-07-23 10:27:39,489 - distilbertfpt-1 - INFO - [Epoch 1/4] Iteration 54856 -> Train Loss: 0.0704, Train Acc: 0.5747, Val Acc:0.38546458141674333(419/1087)
2022-07-23 11:12:35,066 - distilbertfpt-1 - INFO - [Epoch 1/4] Iteration 68570 -> Train Loss: 0.0716, Train Acc: 0.5672, Val Acc:0.3781048758049678(411/1087)
2022-07-23 11:57:39,265 - distilbertfpt-1 - INFO - [Epoch 1/4] Iteration 82284 -> Train Loss: 0.0724, Train Acc: 0.5628, Val Acc:0.37442502299908004(407/1087)
2022-07-23 12:42:22,671 - distilbertfpt-1 - INF

==> save_model : ../../data11/model/distilbert/dbmc-kowiki-202206-1line/batch:32-ep:4-lr:0.000020000-7m23d-13:27


2022-07-23 14:11:57,481 - distilbertfpt-1 - INFO - [Epoch 1/4] Iteration 123426 -> Train Loss: 0.0738, Train Acc: 0.5570, Val Acc:0.37994480220791166(413/1087)
2022-07-23 14:56:41,876 - distilbertfpt-1 - INFO - [Epoch 1/4] Iteration 137140 -> Train Loss: 0.0740, Train Acc: 0.5554, Val Acc:0.3891444342226311(423/1087)


...

  0%|          | 0/137142 [00:00<?, ?it/s]

2022-07-23 23:10:44,260 - distilbertfpt-1 - INFO - [Epoch 3/4] Iteration 287994 -> Train Loss: 0.0731, Train Acc: 0.5567, Val Acc:0.3845446182152714(418/1087)
2022-07-23 23:55:18,345 - distilbertfpt-1 - INFO - [Epoch 3/4] Iteration 301708 -> Train Loss: 0.0549, Train Acc: 0.6528, Val Acc:0.39098436062557496(425/1087)
2022-07-24 00:40:10,651 - distilbertfpt-1 - INFO - [Epoch 3/4] Iteration 315422 -> Train Loss: 0.0555, Train Acc: 0.6513, Val Acc:0.37534498620055196(408/1087)
2022-07-24 01:25:01,274 - distilbertfpt-1 - INFO - [Epoch 3/4] Iteration 329136 -> Train Loss: 0.0558, Train Acc: 0.6489, Val Acc:0.37258509659613614(405/1087)


==> save_model : ../../data11/model/distilbert/dbmc-kowiki-202206-1line/batch:32-ep:4-lr:0.000020000-7m24d-1:25


2022-07-24 02:09:50,203 - distilbertfpt-1 - INFO - [Epoch 3/4] Iteration 342850 -> Train Loss: 0.0603, Train Acc: 0.6251, Val Acc:0.37534498620055196(408/1087)
2022-07-24 02:54:37,913 - distilbertfpt-1 - INFO - [Epoch 3/4] Iteration 356564 -> Train Loss: 0.0604, Train Acc: 0.6243, Val Acc:0.38086476540938363(414/1087)
2022-07-24 03:39:15,463 - distilbertfpt-1 - INFO - [Epoch 3/4] Iteration 370278 -> Train Loss: 0.0607, Train Acc: 0.6232, Val Acc:0.38362465501379944(417/1087)
2022-07-24 04:23:57,614 - distilbertfpt-1 - INFO - [Epoch 3/4] Iteration 383992 -> Train Loss: 0.0608, Train Acc: 0.6238, Val Acc:0.37994480220791166(413/1087)
2022-07-24 05:08:42,538 - distilbertfpt-1 - INFO - [Epoch 3/4] Iteration 397706 -> Train Loss: 0.0609, Train Acc: 0.6225, Val Acc:0.3781048758049678(411/1087)


OSError: [Errno 12] Cannot allocate memory

In [ ]:
# 그래프로 loss 표기
#!pip install matplotlib
import matplotlib.pyplot as plt

plt.plot(list_train_loss, label='Train Loss')
#plt.plot(list_train_acc, label='Train Accuracy')
#plt.plot(list_validation_acc, label='Eval Accuracy')
plt.legend()

plt.show()

plt.plot(list_train_acc, label='Train Accuracy')
plt.plot(list_validation_acc, label='Eval Accuracy')
plt.legend()
plt.show()

In [ ]:
### 전체모델 저장
SaveBERTModel(model, tokenizer, OUTPATH, epochs, learning_rate, batch_size)